# Documentación del flujo pensiones_basico


Paso 1: Importamos las librerías necesarias para descarga, limpieza y análisis.


In [1]:
%matplotlib inline

import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import entropy as shannon_entropy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.inspection import permutation_importance

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot


Paso 2: Definimos la fuente del API y descargamos todas las páginas del dataset.


In [2]:
BASE = "https://www.datos.gov.co"
RESOURCE = "uawh-cjvi"
URL = f"{BASE}/resource/{RESOURCE}.json"

try:
    total_filas = int(requests.get(f"{URL}?$select=count(*)", timeout=60).json()[0]["count"])
except Exception:
    total_filas = None

print("Total reportado:", total_filas)

Lista_paginas = []
limit = 50000
offset = 0

while True:
    params = {"$limit": limit, "$offset": offset}
    r = requests.get(URL, params=params, timeout=120)
    r.raise_for_status()
    pagina = r.json()
    if not pagina:
        break
    Lista_paginas.append(pd.DataFrame(pagina))
    offset += limit
    print(f"Descargadas: {offset} filas…")
    time.sleep(0.3)

if Lista_paginas:
    df = pd.concat(Lista_paginas, ignore_index=True)
else:
    df = pd.DataFrame()


Total reportado: 89769
Descargadas: 50000 filas…
Descargadas: 100000 filas…


Paso 3: Convertimos las columnas clave a tipos adecuados y rellenamos valores faltantes.


In [3]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")

df["valor_unidad"] = (
    df["valor_unidad"]
      .astype(str)
      .str.replace(r"[^\d\-,\.]", "", regex=True)
      .str.replace(",", ".", regex=False)
      .astype(float)
)

print(df.dtypes)
print(df.isnull().sum())

df["valor_unidad"] = df["valor_unidad"].ffill()
df["valor_unidad"] = df["valor_unidad"].interpolate()


fecha                datetime64[ns]
codigo_entidad               object
nombre_entidad               object
codigo_patrimonio            object
nombre_fondo                 object
valor_unidad                float64
dtype: object
fecha                0
codigo_entidad       0
nombre_entidad       0
codigo_patrimonio    0
nombre_fondo         0
valor_unidad         0
dtype: int64


Paso 4: Exploramos nulos, cardinalidad y distribuciones básicas de entidades y fondos.


In [4]:
nulls = df.isna().mean().sort_values(ascending=False).mul(100).round(2)
print(nulls)

cardinalidad = df.nunique(dropna=True).sort_values(ascending=False)
print(cardinalidad)

print("Valores únicos en nombre_entidad:", df["nombre_entidad"].dropna().unique()[:10])
print("Valores únicos en nombre_fondo:", df["nombre_fondo"].dropna().unique()[:10])
print("Conteo nombre_entidad:")
print(df["nombre_entidad"].value_counts(dropna=False).head(10))
print("Conteo nombre_fondo:")
print(df["nombre_fondo"].value_counts(dropna=False).head(20))


fecha                0.0
codigo_entidad       0.0
nombre_entidad       0.0
codigo_patrimonio    0.0
nombre_fondo         0.0
valor_unidad         0.0
dtype: float64
valor_unidad         88973
fecha                 3591
codigo_patrimonio        7
nombre_fondo             7
codigo_entidad           4
nombre_entidad           4
dtype: int64
Valores únicos en nombre_entidad: ['"Proteccion"' '"Porvenir"' 'Skandia Afp - Accai S.A.'
 '"Colfondos S.A." Y "Colfondos"']
Valores únicos en nombre_fondo: ['Fondo de Cesantias Largo Plazo' 'Fondo de Cesantias Corto Plazo'
 'Fondo de Pensiones Moderado' 'Fondo de Pensiones Conservador'
 'Fondo de Pensiones Mayor Riesgo' 'Fondo de Pensiones Retiro Programado'
 'Fondo de Pensiones Alternativo']
Conteo nombre_entidad:
nombre_entidad
Skandia Afp - Accai S.A.          25137
"Porvenir"                        21546
"Colfondos S.A." Y "Colfondos"    21546
"Proteccion"                      21540
Name: count, dtype: int64
Conteo nombre_fondo:
nombre_fondo
Fondo

Paso 5: Exportamos versiones limpias y diccionarios de referencia a CSV.


In [5]:
Path("data/raw").mkdir(parents=True, exist_ok=True)

df_clean = df.drop(columns=["codigo_entidad", "codigo_patrimonio"], errors="ignore")
df_clean.to_csv("data/raw/pensionesLimpio.csv", index=False)

if "codigo_entidad" in df.columns:
    dict_entidad = (
        df[["nombre_entidad", "codigo_entidad"]]
          .drop_duplicates()
          .set_index("nombre_entidad")["codigo_entidad"]
          .to_dict()
    )
    df[["nombre_entidad", "codigo_entidad"]].drop_duplicates().to_csv(
        "data/raw/entidad_codigo.csv", index=False
    )
else:
    dict_entidad = {}

if "codigo_patrimonio" in df.columns:
    dict_fondo = (
        df[["nombre_fondo", "codigo_patrimonio"]]
          .drop_duplicates()
          .set_index("nombre_fondo")["codigo_patrimonio"]
          .to_dict()
    )
    df[["nombre_fondo", "codigo_patrimonio"]].drop_duplicates().to_csv(
        "data/raw/fondos_codigo.csv", index=False
    )
else:
    dict_fondo = {}


Paso 6: Normalizamos textos y verificamos relaciones uno a uno entre códigos y nombres.


In [6]:
for columna in ["nombre_entidad", "nombre_fondo"]:
    if columna in df.columns:
        df[columna] = (
            df[columna]
              .astype(str)
              .str.strip()
              .str.replace(r"\s+", " ", regex=True)
        )

print("Cardinalidad después de limpieza:")
print(df[["nombre_entidad", "nombre_fondo"]].nunique())

if "codigo_entidad" in df.columns:
    print("Relación código_entidad → nombre_entidad:")
    print(df.groupby("codigo_entidad")["nombre_entidad"].nunique().sort_values(ascending=False).head())
    print("Relación nombre_entidad → código_entidad:")
    print(df.groupby("nombre_entidad")["codigo_entidad"].nunique().sort_values(ascending=False).head())

if "codigo_patrimonio" in df.columns:
    print("Relación código_patrimonio → nombre_fondo:")
    print(df.groupby("codigo_patrimonio")["nombre_fondo"].nunique().sort_values(ascending=False).head())
    print("Relación nombre_fondo → código_patrimonio:")
    print(df.groupby("nombre_fondo")["codigo_patrimonio"].nunique().sort_values(ascending=False).head())


Cardinalidad después de limpieza:
nombre_entidad    4
nombre_fondo      7
dtype: int64
Relación código_entidad → nombre_entidad:
codigo_entidad
10    1
2     1
3     1
9     1
Name: nombre_entidad, dtype: int64
Relación nombre_entidad → código_entidad:
nombre_entidad
"Colfondos S.A." Y "Colfondos"    1
"Porvenir"                        1
"Proteccion"                      1
Skandia Afp - Accai S.A.          1
Name: codigo_entidad, dtype: int64
Relación código_patrimonio → nombre_fondo:
codigo_patrimonio
1       1
1000    1
2       1
5000    1
6000    1
Name: nombre_fondo, dtype: int64
Relación nombre_fondo → código_patrimonio:
nombre_fondo
Fondo de Cesantias Corto Plazo     1
Fondo de Cesantias Largo Plazo     1
Fondo de Pensiones Alternativo     1
Fondo de Pensiones Conservador     1
Fondo de Pensiones Mayor Riesgo    1
Name: codigo_patrimonio, dtype: int64


Paso 7: Contamos y eliminamos duplicados exactos y conceptuales.


In [7]:
print("=== ANÁLISIS DE DUPLICADOS ===")
duplicados = df.duplicated().sum()
print(f"Filas duplicadas exactas: {duplicados}")
if duplicados > 0:
    df = df.drop_duplicates()
    print(f"Dataset después de eliminar duplicados exactos: {len(df)} filas")
else:
    print("✓ No hay duplicados exactos")

duplicados_conceptuales = df.duplicated(subset=["nombre_entidad", "nombre_fondo", "fecha"]).sum()
print(f"Duplicados entidad-fondo-fecha: {duplicados_conceptuales}")
if duplicados_conceptuales > 0:
    df = df.drop_duplicates(subset=["nombre_entidad", "nombre_fondo", "fecha"], keep="first")
    print(f"Dataset después de limpieza: {len(df)} filas")


=== ANÁLISIS DE DUPLICADOS ===
Filas duplicadas exactas: 0
✓ No hay duplicados exactos
Duplicados entidad-fondo-fecha: 0


Paso 8: Detectamos outliers con IQR y etiquetamos la columna es_outlier.


In [8]:
Q1 = df["valor_unidad"].quantile(0.25)
Q3 = df["valor_unidad"].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

outliers = df[(df["valor_unidad"] < limite_inferior) | (df["valor_unidad"] > limite_superior)]

print(f"Límite inferior (outliers): {limite_inferior:.2f}")
print(f"Límite superior (outliers): {limite_superior:.2f}")
print(f"Total de outliers detectados: {len(outliers)}")

if len(outliers) > 0:
    print(outliers[["nombre_entidad", "nombre_fondo", "fecha", "valor_unidad"]].head())
    df["es_outlier"] = False
    df.loc[outliers.index, "es_outlier"] = True
else:
    df["es_outlier"] = False


Límite inferior (outliers): 3481.55
Límite superior (outliers): 84213.85
Total de outliers detectados: 2475
               nombre_entidad                    nombre_fondo      fecha  \
18   Skandia Afp - Accai S.A.  Fondo de Pensiones Alternativo 2016-01-01   
43   Skandia Afp - Accai S.A.  Fondo de Pensiones Alternativo 2016-01-02   
68   Skandia Afp - Accai S.A.  Fondo de Pensiones Alternativo 2016-01-03   
93   Skandia Afp - Accai S.A.  Fondo de Pensiones Alternativo 2016-01-04   
118  Skandia Afp - Accai S.A.  Fondo de Pensiones Alternativo 2016-01-05   

     valor_unidad  
18        2637.30  
43        2637.63  
68        2637.93  
93        2629.92  
118       2627.32  


Paso 9: Optimizamos tipos y generamos variables temporales y tipo_fondo.


In [9]:
df["nombre_entidad"] = df["nombre_entidad"].astype("category")
df["nombre_fondo"] = df["nombre_fondo"].astype("category")
df["es_outlier"] = df["es_outlier"].astype(bool)

df["año"] = df["fecha"].dt.year
df["mes"] = df["fecha"].dt.month
df["trimestre"] = df["fecha"].dt.quarter

def clasificar_fondo(nombre):
    nombre = str(nombre).lower()
    if "cesantia" in nombre:
        return "Cesantías"
    if "pension" in nombre:
        return "Pensiones"
    if "alternativo" in nombre:
        return "Alternativo"
    return "Otros"

df["tipo_fondo"] = df["nombre_fondo"].apply(clasificar_fondo).astype("category")
print(df[["año", "mes", "trimestre", "tipo_fondo"]].head())


    año  mes  trimestre tipo_fondo
0  2016    1          1  Cesantías
1  2016    1          1  Cesantías
2  2016    1          1  Pensiones
3  2016    1          1  Pensiones
4  2016    1          1  Pensiones


Paso 10: Definimos guardar_subset y exportamos subconjuntos por entidad y fondo.


In [10]:
def guardar_subset(dataframe, columna, valores, ruta_salida):
    if isinstance(valores, (list, tuple, set)):
        df_subset = dataframe.loc[dataframe[columna].isin(valores)].copy()
    else:
        df_subset = dataframe.loc[dataframe[columna].eq(valores)].copy()
    if columna in df_subset.columns:
        df_subset = df_subset.drop(columns=[columna])
    df_subset.to_csv(ruta_salida, index=False)
    print(ruta_salida, df_subset.shape)
    return df_subset

# Subconjuntos Skandia
df_skandia = guardar_subset(df, "nombre_entidad", "Skandia Afp - Accai S.A.", "data/raw/skandia.csv")
df_skandia_cesantias_largo_plazo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Cesantias Largo Plazo", "data/raw/skandia_fondo_cesantias_largo_plazo.csv")
df_skandia_cesantias_corto_plazo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Cesantias Corto Plazo", "data/raw/skandia_fondo_cesantias_corto_plazo.csv")
df_skandia_pensiones_moderado = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Moderado", "data/raw/skandia_fondo_pensiones_moderado.csv")
df_skandia_pensiones_conservador = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Conservador", "data/raw/skandia_fondo_pensiones_conservador.csv")
df_skandia_pensiones_mayor_riesgo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Mayor Riesgo", "data/raw/skandia_fondo_pensiones_mayor_riesgo.csv")
df_skandia_pensiones_retiro_programado = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Retiro Programado", "data/raw/skandia_fondo_pensiones_retiro_programado.csv")
df_skandia_pensiones_alternativo = guardar_subset(df_skandia, "nombre_fondo", "Fondo de Pensiones Alternativo", "data/raw/skandia_fondo_pensiones_alternativo.csv")

# Subconjuntos Protección
df_proteccion = guardar_subset(df, "nombre_entidad", '"Proteccion"', "data/raw/proteccion.csv")
df_proteccion_cesantias_largo_plazo = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Cesantias Largo Plazo", "data/raw/proteccion_fondo_cesantias_largo_plazo.csv")
df_proteccion_cesantias_corto_plazo = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Cesantias Corto Plazo", "data/raw/proteccion_fondo_cesantias_corto_plazo.csv")
df_proteccion_pensiones_moderado = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Pensiones Moderado", "data/raw/proteccion_fondo_pensiones_moderado.csv")
df_proteccion_pensiones_conservador = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Pensiones Conservador", "data/raw/proteccion_fondo_pensiones_conservador.csv")
df_proteccion_pensiones_mayor_riesgo = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Pensiones Mayor Riesgo", "data/raw/proteccion_fondo_pensiones_mayor_riesgo.csv")
df_proteccion_pensiones_retiro_programado = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Pensiones Retiro Programado", "data/raw/proteccion_fondo_pensiones_retiro_programado.csv")
df_proteccion_pensiones_alternativo = guardar_subset(df_proteccion, "nombre_fondo", "Fondo de Pensiones Alternativo", "data/raw/proteccion_fondo_pensiones_alternativo.csv")

# Subconjuntos Porvenir
df_porvenir = guardar_subset(df, "nombre_entidad", '"Porvenir"', "data/raw/porvenir.csv")
df_porvenir_cesantias_largo_plazo = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Cesantias Largo Plazo", "data/raw/porvenir_fondo_cesantias_largo_plazo.csv")
df_porvenir_cesantias_corto_plazo = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Cesantias Corto Plazo", "data/raw/porvenir_fondo_cesantias_corto_plazo.csv")
df_porvenir_pensiones_moderado = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Pensiones Moderado", "data/raw/porvenir_fondo_pensiones_moderado.csv")
df_porvenir_pensiones_conservador = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Pensiones Conservador", "data/raw/porvenir_fondo_pensiones_conservador.csv")
df_porvenir_pensiones_mayor_riesgo = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Pensiones Mayor Riesgo", "data/raw/porvenir_fondo_pensiones_mayor_riesgo.csv")
df_porvenir_pensiones_retiro_programado = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Pensiones Retiro Programado", "data/raw/porvenir_fondo_pensiones_retiro_programado.csv")
df_porvenir_pensiones_alternativo = guardar_subset(df_porvenir, "nombre_fondo", "Fondo de Pensiones Alternativo", "data/raw/porvenir_fondo_pensiones_alternativo.csv")

# Subconjuntos Colfondos
df_colfondos = guardar_subset(df, "nombre_entidad", '"Colfondos S.A." Y "Colfondos"', "data/raw/colfondos.csv")
df_colfondos_cesantias_largo_plazo = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Cesantias Largo Plazo", "data/raw/colfondos_fondo_cesantias_largo_plazo.csv")
df_colfondos_cesantias_corto_plazo = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Cesantias Corto Plazo", "data/raw/colfondos_fondo_cesantias_corto_plazo.csv")
df_colfondos_pensiones_moderado = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Pensiones Moderado", "data/raw/colfondos_fondo_pensiones_moderado.csv")
df_colfondos_pensiones_conservador = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Pensiones Conservador", "data/raw/colfondos_fondo_pensiones_conservador.csv")
df_colfondos_pensiones_mayor_riesgo = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Pensiones Mayor Riesgo", "data/raw/colfondos_fondo_pensiones_mayor_riesgo.csv")
df_colfondos_pensiones_retiro_programado = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Pensiones Retiro Programado", "data/raw/colfondos_fondo_pensiones_retiro_programado.csv")
df_colfondos_pensiones_alternativo = guardar_subset(df_colfondos, "nombre_fondo", "Fondo de Pensiones Alternativo", "data/raw/colfondos_fondo_pensiones_alternativo.csv")


data/raw/skandia.csv (25137, 10)
data/raw/skandia_fondo_cesantias_largo_plazo.csv (3591, 9)
data/raw/skandia_fondo_cesantias_corto_plazo.csv (3591, 9)
data/raw/skandia_fondo_pensiones_moderado.csv (3591, 9)
data/raw/skandia_fondo_pensiones_conservador.csv (3591, 9)
data/raw/skandia_fondo_pensiones_mayor_riesgo.csv (3591, 9)
data/raw/skandia_fondo_pensiones_retiro_programado.csv (3591, 9)
data/raw/skandia_fondo_pensiones_alternativo.csv (3591, 9)
data/raw/proteccion.csv (21540, 10)
data/raw/proteccion_fondo_cesantias_largo_plazo.csv (3590, 9)
data/raw/proteccion_fondo_cesantias_corto_plazo.csv (3590, 9)
data/raw/proteccion_fondo_pensiones_moderado.csv (3590, 9)
data/raw/proteccion_fondo_pensiones_conservador.csv (3590, 9)
data/raw/proteccion_fondo_pensiones_mayor_riesgo.csv (3590, 9)
data/raw/proteccion_fondo_pensiones_retiro_programado.csv (3590, 9)
data/raw/proteccion_fondo_pensiones_alternativo.csv (0, 9)
data/raw/porvenir.csv (21546, 10)
data/raw/porvenir_fondo_cesantias_largo_plazo

Paso 11: Creamos gráficos comparativos por tipo de fondo entre entidades.


In [11]:
def graficar_comparacion_entidades_por_fondo(fondos_a_comparar, titulo_base, nombre_archivo):
    plt.figure(figsize=(14, 8))
    colores = ['blue', 'red', 'green', 'orange', 'purple', 'brown']
    for i, (entidad, df_fondo) in enumerate(fondos_a_comparar.items()):
        if len(df_fondo) > 0:
            color = colores[i % len(colores)]
            plt.plot(df_fondo['fecha'], df_fondo['valor_unidad'], label=entidad, color=color, linewidth=2, alpha=0.8)
    plt.title(f'{titulo_base} - Comparación por Entidad')
    plt.xlabel('Fecha')
    plt.ylabel('Valor Unidad')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)
    plt.savefig(f"data/graficas_comparativas/{nombre_archivo}.png", dpi=300, bbox_inches='tight')
    plt.close()

fondos_moderado = {
    'Skandia': df_skandia_pensiones_moderado,
    'Protección': df_proteccion_pensiones_moderado,
    'Porvenir': df_porvenir_pensiones_moderado,
    'Colfondos': df_colfondos_pensiones_moderado
}
graficar_comparacion_entidades_por_fondo(fondos_moderado, "Fondo de Pensiones Moderado", "comparacion_pensiones_moderado")

fondos_conservador = {
    'Skandia': df_skandia_pensiones_conservador,
    'Protección': df_proteccion_pensiones_conservador,
    'Porvenir': df_porvenir_pensiones_conservador,
    'Colfondos': df_colfondos_pensiones_conservador
}
graficar_comparacion_entidades_por_fondo(fondos_conservador, "Fondo de Pensiones Conservador", "comparacion_pensiones_conservador")

fondos_cesantias_largo = {
    'Skandia': df_skandia_cesantias_largo_plazo,
    'Protección': df_proteccion_cesantias_largo_plazo,
    'Porvenir': df_porvenir_cesantias_largo_plazo,
    'Colfondos': df_colfondos_cesantias_largo_plazo
}
graficar_comparacion_entidades_por_fondo(fondos_cesantias_largo, "Fondo de Cesantías Largo Plazo", "comparacion_cesantias_largo")

fondos_cesantias_corto = {
    'Skandia': df_skandia_cesantias_corto_plazo,
    'Protección': df_proteccion_cesantias_corto_plazo,
    'Porvenir': df_porvenir_cesantias_corto_plazo,
    'Colfondos': df_colfondos_cesantias_corto_plazo
}
graficar_comparacion_entidades_por_fondo(fondos_cesantias_corto, "Fondo de Cesantías Corto Plazo", "comparacion_cesantias_corto")

fondos_mayor_riesgo = {
    'Skandia': df_skandia_pensiones_mayor_riesgo,
    'Protección': df_proteccion_pensiones_mayor_riesgo,
    'Porvenir': df_porvenir_pensiones_mayor_riesgo,
    'Colfondos': df_colfondos_pensiones_mayor_riesgo
}
graficar_comparacion_entidades_por_fondo(fondos_mayor_riesgo, "Fondo de Pensiones Mayor Riesgo", "comparacion_pensiones_mayor_riesgo")

fondos_retiro_programado = {
    'Skandia': df_skandia_pensiones_retiro_programado,
    'Protección': df_proteccion_pensiones_retiro_programado,
    'Porvenir': df_porvenir_pensiones_retiro_programado,
    'Colfondos': df_colfondos_pensiones_retiro_programado
}
graficar_comparacion_entidades_por_fondo(fondos_retiro_programado, "Fondo de Pensiones retiro programado", "comparacion_pensiones_retiro_programado")

fondos_alternativo = {
    'Skandia': df_skandia_pensiones_alternativo,
    'Protección': df_proteccion_pensiones_alternativo,
    'Porvenir': df_porvenir_pensiones_alternativo,
    'Colfondos': df_colfondos_pensiones_alternativo
}
graficar_comparacion_entidades_por_fondo(fondos_alternativo, "Fondo de Pensiones alternativo", "comparacion_pensiones_alternativo")


Paso 12: Analizamos evolución y correlaciones por entidad con funciones auxiliares.


In [12]:
def evolucion_todos_fondos_entidad(entidad_nombre, dataframes_fondos):
    plt.figure(figsize=(16, 10))
    fondos_colores = {
        'Cesantías Largo Plazo': 'blue',
        'Cesantías Corto Plazo': 'lightblue',
        'Pensiones Moderado': 'green',
        'Pensiones Conservador': 'darkgreen',
        'Pensiones Mayor Riesgo': 'red',
        'Pensiones Retiro Programado': 'orange',
        'Pensiones Alternativo': 'purple'
    }
    for fondo_nombre, color in fondos_colores.items():
        if fondo_nombre in dataframes_fondos and len(dataframes_fondos[fondo_nombre]) > 0:
            df_temp = dataframes_fondos[fondo_nombre]
            valor_base = df_temp['valor_unidad'].iloc[0]
            df_normalizado = (df_temp['valor_unidad'] / valor_base * 100)
            plt.plot(df_temp['fecha'], df_normalizado, label=fondo_nombre, color=color, linewidth=2, alpha=0.7)
    plt.title(f'Evolución de Todos los Fondos - {entidad_nombre} (Base 100)')
    plt.xlabel('Fecha')
    plt.ylabel('Valor Normalizado')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)
    plt.savefig(f'data/graficas_comparativas/evolucion_todos_fondos_{entidad_nombre.lower()}.png', dpi=300, bbox_inches='tight')
    plt.close()

def matriz_correlacion_fondos(entidad_nombre, dataframes_fondos):
    datos_correlacion = {}
    for fondo_nombre, df_temp in dataframes_fondos.items():
        if len(df_temp) > 0:
            serie = df_temp.set_index('fecha')['valor_unidad'].sort_index()
            returns = serie.pct_change().dropna()
            datos_correlacion[fondo_nombre] = returns
    df_correlacion = pd.DataFrame(datos_correlacion)
    matriz_corr = df_correlacion.corr()
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(matriz_corr, dtype=bool))
    sns.heatmap(matriz_corr, mask=mask, annot=True, cmap='coolwarm', center=0, square=True, fmt='.2f', cbar_kws={'label': 'Coeficiente'})
    plt.title(f'Matriz de Correlación - {entidad_nombre} (returns diarios)')
    plt.tight_layout()
    Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)
    plt.savefig(f'data/graficas_comparativas/correlacion_{entidad_nombre.lower()}.png', dpi=300, bbox_inches='tight')
    plt.close()
    return matriz_corr

entidades_dataframes = {
    'Skandia': {
        'Cesantías Largo Plazo': df_skandia_cesantias_largo_plazo,
        'Cesantías Corto Plazo': df_skandia_cesantias_corto_plazo,
        'Pensiones Moderado': df_skandia_pensiones_moderado,
        'Pensiones Conservador': df_skandia_pensiones_conservador,
        'Pensiones Mayor Riesgo': df_skandia_pensiones_mayor_riesgo,
        'Pensiones Retiro Programado': df_skandia_pensiones_retiro_programado,
        'Pensiones Alternativo': df_skandia_pensiones_alternativo
    },
    'Protección': {
        'Cesantías Largo Plazo': df_proteccion_cesantias_largo_plazo,
        'Cesantías Corto Plazo': df_proteccion_cesantias_corto_plazo,
        'Pensiones Moderado': df_proteccion_pensiones_moderado,
        'Pensiones Conservador': df_proteccion_pensiones_conservador,
        'Pensiones Mayor Riesgo': df_proteccion_pensiones_mayor_riesgo,
        'Pensiones Retiro Programado': df_proteccion_pensiones_retiro_programado,
        'Pensiones Alternativo': df_proteccion_pensiones_alternativo
    },
    'Porvenir': {
        'Cesantías Largo Plazo': df_porvenir_cesantias_largo_plazo,
        'Cesantías Corto Plazo': df_porvenir_cesantias_corto_plazo,
        'Pensiones Moderado': df_porvenir_pensiones_moderado,
        'Pensiones Conservador': df_porvenir_pensiones_conservador,
        'Pensiones Mayor Riesgo': df_porvenir_pensiones_mayor_riesgo,
        'Pensiones Retiro Programado': df_porvenir_pensiones_retiro_programado,
        'Pensiones Alternativo': df_porvenir_pensiones_alternativo
    },
    'Colfondos': {
        'Cesantías Largo Plazo': df_colfondos_cesantias_largo_plazo,
        'Cesantías Corto Plazo': df_colfondos_cesantias_corto_plazo,
        'Pensiones Moderado': df_colfondos_pensiones_moderado,
        'Pensiones Conservador': df_colfondos_pensiones_conservador,
        'Pensiones Mayor Riesgo': df_colfondos_pensiones_mayor_riesgo,
        'Pensiones Retiro Programado': df_colfondos_pensiones_retiro_programado,
        'Pensiones Alternativo': df_colfondos_pensiones_alternativo
    }
}

evolucion_todos_fondos_entidad('Skandia', entidades_dataframes['Skandia'])
evolucion_todos_fondos_entidad('Protección', entidades_dataframes['Protección'])
evolucion_todos_fondos_entidad('Porvenir', entidades_dataframes['Porvenir'])
evolucion_todos_fondos_entidad('Colfondos', entidades_dataframes['Colfondos'])

matriz_correlacion_fondos('Skandia', entidades_dataframes['Skandia'])
matriz_correlacion_fondos('Protección', entidades_dataframes['Protección'])
matriz_correlacion_fondos('Porvenir', entidades_dataframes['Porvenir'])
matriz_correlacion_fondos('Colfondos', entidades_dataframes['Colfondos'])


,Cesantías Largo Plazo,Cesantías Corto Plazo,Pensiones Moderado,Pensiones Conservador,Pensiones Mayor Riesgo,Pensiones Retiro Programado
Cesantías Largo Plazo,1.000000,0.124827,0.955504,0.868332,0.944808,0.748912
Cesantías Corto Plazo,0.124827,1.000000,0.115030,0.222580,0.089369,0.216842
Pensiones Moderado,0.955504,0.115030,1.000000,0.834002,0.942904,0.737826
Pensiones Conservador,0.868332,0.222580,0.834002,1.000000,0.771402,0.884027
Pensiones Mayor Riesgo,0.944808,0.089369,0.942904,0.771402,1.000000,0.649172
Pensiones Retiro Programado,0.748912,0.216842,0.737826,0.884027,0.649172,1.000000


Paso 13: Generamos columnas lag para comparaciones temporales de valor_unidad.


In [13]:
for lag in range(1, len(df) // 30, 30):
    df[f'lag_{lag}'] = df['valor_unidad'].shift(lag)

lags = [1, 7, 30, 90, 180, 365]
lag_columns = {f'lag_{lag}': df['valor_unidad'].shift(lag) for lag in lags}
lag_df = pd.DataFrame(lag_columns)
df = pd.concat([df, lag_df], axis=1)


/tmp/ipykernel_143348/1063748102.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{lag}'] = df['valor_unidad'].shift(lag)
/tmp/ipykernel_143348/1063748102.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{lag}'] = df['valor_unidad'].shift(lag)
/tmp/ipykernel_143348/1063748102.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a d

Paso 14: Ejecutamos el EDA completo con estadísticas, gráficas y descomposición.


In [14]:
Path("data/graficas_comparativas").mkdir(parents=True, exist_ok=True)

print("=== ANÁLISIS EXPLORATORIO COMPLETO (EDA) ===")
print(df[['valor_unidad']].describe())
print(df.groupby('tipo_fondo', observed=True)['valor_unidad'].describe())

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
df['valor_unidad'].hist(bins=50, alpha=0.7)
plt.title('Distribución de valor_unidad')
plt.subplot(1, 2, 2)
df['valor_unidad'].plot(kind='density')
plt.title('Densidad de valor_unidad')
plt.tight_layout()
plt.savefig('data/graficas_comparativas/distribucion_y_densidad_valor_unidad.png', dpi=300, bbox_inches='tight')
plt.close()

evolucion_anual = df.groupby('año')['valor_unidad'].agg(['mean', 'std', 'min', 'max'])
print(evolucion_anual)

plt.figure(figsize=(12, 6))
plt.plot(evolucion_anual.index, evolucion_anual['mean'], marker='o')
plt.fill_between(evolucion_anual.index, evolucion_anual['mean'] - evolucion_anual['std'], evolucion_anual['mean'] + evolucion_anual['std'], alpha=0.2)
plt.title('Evolución anual del valor unidad (media ± desviación)')
plt.grid(True, alpha=0.3)
plt.savefig('data/graficas_comparativas/evolucion_anual.png', dpi=300, bbox_inches='tight')
plt.close()

fondo_ejemplo = df_skandia_pensiones_moderado.set_index('fecha')['valor_unidad']

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plot_acf(fondo_ejemplo, lags=30, ax=plt.gca())
plt.subplot(1, 2, 2)
plot_pacf(fondo_ejemplo, lags=30, ax=plt.gca())
plt.tight_layout()
plt.savefig('data/graficas_comparativas/autocorrelacion.png', dpi=300, bbox_inches='tight')
plt.close()

fondo_mensual = fondo_ejemplo.resample('ME').mean()
try:
    descomposicion = seasonal_decompose(fondo_mensual, model='additive', period=12)
    fig = descomposicion.plot()
    fig.set_size_inches(12, 8)
    fig.suptitle('Descomposición estacional - Fondo moderado Skandia', fontsize=14)
    plt.savefig('data/graficas_comparativas/descomposicion_estacional.png', dpi=300, bbox_inches='tight')
    plt.close()
except Exception as e:
    print('Error en descomposición estacional:', e)

estacionalidad_mensual = df.groupby('mes')['valor_unidad'].mean()
plt.figure(figsize=(10, 6))
estacionalidad_mensual.plot(kind='bar', color='skyblue', alpha=0.7)
plt.title('Comportamiento estacional promedio por mes')
plt.savefig('data/graficas_comparativas/estacionalidad_mensual.png', dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='tipo_fondo', y='valor_unidad')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('data/graficas_comparativas/boxplot_tipos_fondo.png', dpi=300, bbox_inches='tight')
plt.close()

fondo_ejemplo_vol = fondo_ejemplo.rolling(window=30).std()
plt.figure(figsize=(12, 6))
plt.plot(fondo_ejemplo_vol.index, fondo_ejemplo_vol.values, color='red', alpha=0.7)
plt.title('Volatilidad rolling (30 días) - Fondo moderado Skandia')
plt.savefig('data/graficas_comparativas/volatilidad_rolling.png', dpi=300, bbox_inches='tight')
plt.close()

pivot_corr = df.pivot_table(index='fecha', columns='tipo_fondo', values='valor_unidad', observed=False).corr()
plt.figure(figsize=(8, 6))
sns.heatmap(pivot_corr, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlación entre tipos de fondo')
plt.tight_layout()
plt.savefig('data/graficas_comparativas/correlacion_tipos_fondo.png', dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(12, 6))
plt.plot(fondo_ejemplo.index, fondo_ejemplo.values, label='Valor diario', alpha=0.3)
plt.plot(fondo_ejemplo.rolling(30).mean(), label='Media 30 días', linewidth=2)
plt.plot(fondo_ejemplo.rolling(90).mean(), label='Media 90 días', linewidth=2)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('data/graficas_comparativas/analisis_tendencia.png', dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(12, 6))
plt.scatter(df.index, df['valor_unidad'], c=df['es_outlier'], cmap='coolwarm', alpha=0.6)
plt.title('Identificación visual de outliers')
plt.savefig('data/graficas_comparativas/outliers_detallado.png', dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(12, 6))
for tipo in df['tipo_fondo'].unique():
    subset = df[df['tipo_fondo'] == tipo]
    plt.hist(subset['valor_unidad'], bins=50, alpha=0.5, label=tipo, density=True)
plt.legend()
plt.title('Distribución de densidad por tipo de fondo')
plt.savefig('data/graficas_comparativas/densidad_tipos_fondo.png', dpi=300, bbox_inches='tight')
plt.close()

years = sorted(df['año'].dropna().unique())
n_cols = 3
n_rows = (len(years) + n_cols - 1) // n_cols
Path('images/analisis_exploratorio').mkdir(parents=True, exist_ok=True)
plt.figure(figsize=(15, 5 * n_rows))
for i, year in enumerate(years, 1):
    plt.subplot(n_rows, n_cols, i)
    data_year = df[df['año'] == year]
    for fondo in data_year['tipo_fondo'].unique():
        data_fondo = data_year[data_year['tipo_fondo'] == fondo]
        monthly_avg = data_fondo.groupby('mes')['valor_unidad'].mean()
        plt.plot(monthly_avg.index, monthly_avg.values, marker='o', label=fondo)
    plt.title(f'Evolución mensual {int(year)}')
    plt.xticks(range(1, 13))
    plt.grid(True, alpha=0.3)
    if i == 1:
        plt.legend()
plt.tight_layout()
plt.savefig('images/analisis_exploratorio/evolucion_mensual_por_año.png', dpi=300, bbox_inches='tight')
plt.close()

print("EDA completado")


=== ANÁLISIS EXPLORATORIO COMPLETO (EDA) ===
       valor_unidad
count  89769.000000
mean   44231.791309
std    16417.473065
min     2596.400000
25%    33756.164041
50%    42634.660000
75%    53939.240000
max    97821.910000
              count          mean           std       min        25%  \
tipo_fondo                                                             
Cesantías   28726.0  34101.995393   6832.263056  21589.17  28804.325   
Pensiones   61043.0  48998.734570  17418.228534   2596.40  40042.200   

                 50%       75%       max  
tipo_fondo                                
Cesantías   33367.67  38213.49  56380.17  
Pensiones   48480.09  59657.30  97821.91  
              mean           std      min       max
año                                                
2016  30933.310505   8090.679461  2596.40  44298.29
2017  34684.921141   9449.220153  2848.70  49635.24
2018  36858.466367  10147.320209  3135.23  51343.60
2019  39836.324999  11367.830222  3263.69  58638.94
20

Paso 15: Guardamos datasets procesados y listos para modelado con resumen.


In [15]:
Path("data/processed").mkdir(parents=True, exist_ok=True)

df.to_csv("data/processed/pensiones_limpio_final.csv", index=False, encoding='utf-8')

resumen_limpieza = {
    'filas_finales': len(df),
    'columnas_finales': len(df.columns),
    'duplicados_eliminados': int(duplicados),
    'outliers_detectados': int(len(outliers)),
    'memoria_mb': df.memory_usage(deep=True).sum() / 1024**2,
    'fecha_limpieza': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}
pd.Series(resumen_limpieza).to_csv("data/processed/resumen_limpieza.csv")

df_modelado = df.dropna(subset=['valor_unidad', 'fecha', 'nombre_entidad', 'nombre_fondo']).copy()
df_modelado.to_csv("data/processed/pensiones_listo_modelado.csv", index=False, encoding='utf-8')
print("✓ Exportes procesados listos")


✓ Exportes procesados listos


Paso 16: Construimos y ejecutamos el pipeline de modelado ARIMA con evaluación.


In [16]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

def analizar_estacionariedad(serie, nombre_serie=""):
    print(f"
--- Análisis de Estacionariedad: {nombre_serie} ---")
    resultado_adf = adfuller(serie.dropna())
    metricas = {
        'estadistico_adf': resultado_adf[0],
        'p_valor': resultado_adf[1],
        'valores_criticos': resultado_adf[4],
        'es_estacionaria': resultado_adf[1] < 0.05
    }
    print(f"Estadístico ADF: {metricas['estadistico_adf']:.4f}")
    print(f"P-valor: {metricas['p_valor']:.4f}")
    if metricas['es_estacionaria']:
        print("✓ La serie ES estacionaria (p-valor < 0.05)")
    else:
        print("✗ La serie NO es estacionaria (p-valor > 0.05)")
        print("  Se requiere diferenciación para modelado ARIMA")
    return metricas

def entrenar_modelo_arima(serie, orden, nombre_serie=""):
    print(f"Entrenando ARIMA{orden} para {nombre_serie}...")
    try:
        modelo = ARIMA(serie, order=orden)
        modelo_ajustado = modelo.fit()
        metricas = {
            'aic': modelo_ajustado.aic,
            'bic': modelo_ajustado.bic,
            'residuos_media': modelo_ajustado.resid.mean(),
            'residuos_std': modelo_ajustado.resid.std()
        }
        print(f"✓ ARIMA{orden} entrenado exitosamente")
        print(f"  AIC: {metricas['aic']:.2f}, BIC: {metricas['bic']:.2f}")
        return modelo_ajustado, metricas
    except Exception as e:
        print(f"✗ Error entrenando ARIMA{orden}: {e}")
        return None, None

def buscar_mejor_arima(serie, parametros_a_probar, nombre_serie=""):
    print(f"
Buscando mejor modelo ARIMA para {nombre_serie}...")
    mejores_metricas = {'aic': float('inf')}
    mejor_modelo = None
    mejor_orden = None
    for orden in parametros_a_probar:
        modelo, metricas = entrenar_modelo_arima(serie, orden, nombre_serie)
        if modelo and metricas and metricas['aic'] < mejores_metricas['aic']:
            mejores_metricas = metricas
            mejor_modelo = modelo
            mejor_orden = orden
    if mejor_modelo:
        print(f"
🎯 MEJOR MODELO ENCONTRADO: ARIMA{mejor_orden} con AIC {mejores_metricas['aic']:.2f}")
        return {'modelo': mejor_modelo, 'orden': mejor_orden, 'metricas': mejores_metricas}
    print("✗ No se pudo encontrar un modelo adecuado")
    return None

def evaluar_pronostico(real, pronosticado, nombre_serie=""):
    print(f"
--- Evaluación de Pronósticos: {nombre_serie} ---")
    mse = np.mean((real - pronosticado)**2)
    mae = np.mean(np.abs(real - pronosticado))
    mape = np.mean(np.abs((real - pronosticado) / real)) * 100
    rmse = np.sqrt(mse)
    metricas = {'MSE': mse, 'MAE': mae, 'MAPE': mape, 'RMSE': rmse}
    for k, v in metricas.items():
        print(f"  {k}: {v:.4f}")
    print(f"  Error porcentual promedio (MAPE): {mape:.2f}%")
    return metricas

def pipeline_modelado_completo(df_serie, nombre_serie, columna_valor='valor_unidad'):
    print(f"
{'='*50}
INICIANDO PIPELINE DE MODELADO: {nombre_serie}
{'='*50}")
    resultados = {}
    serie = df_serie.set_index('fecha')[columna_valor].sort_index()
    resultados['serie_original'] = serie.copy()
    resultados['estacionariedad'] = analizar_estacionariedad(serie, nombre_serie)
    if not resultados['estacionariedad']['es_estacionaria']:
        print("
3. ⚙️  Aplicando diferenciación...")
        serie_diff = serie.diff().dropna()
        resultados['serie_diferenciada'] = serie_diff
        resultados['estacionariedad_diff'] = analizar_estacionariedad(serie_diff, f"{nombre_serie} (diferenciada)")
        serie_para_modelar = serie_diff
    else:
        serie_para_modelar = serie
    parametros_a_probar = [(1,0,0), (1,1,1), (2,1,2), (0,1,1), (1,1,0)]
    resultados['mejor_modelo'] = buscar_mejor_arima(serie_para_modelar, parametros_a_probar, nombre_serie)
    if resultados['mejor_modelo']:
        modelo = resultados['mejor_modelo']['modelo']
        train_size = int(len(serie_para_modelar) * 0.8)
        train, test = serie_para_modelar[:train_size], serie_para_modelar[train_size:]
        modelo_train = ARIMA(train, order=resultados['mejor_modelo']['orden']).fit()
        pronostico = modelo_train.forecast(steps=len(test))
        resultados['evaluacion'] = evaluar_pronostico(test.values, pronostico.values, nombre_serie)
        print("
6. 🔮 Generando pronósticos futuros...")
        pronostico_futuro = modelo.forecast(steps=30)
        resultados['pronostico_futuro'] = pronostico_futuro
        print(f"Pronóstico 30 días: tendencia {'al alza' if pronostico_futuro.iloc[-1] > serie_para_modelar.iloc[-1] else 'a la baja'}")
    Path("data/modelos").mkdir(parents=True, exist_ok=True)
    resumen_modelado = {
        'serie': nombre_serie,
        'mejor_modelo': f"ARIMA{resultados.get('mejor_modelo', {}).get('orden', 'N/A')}",
        'aic': resultados.get('mejor_modelo', {}).get('metricas', {}).get('aic', 'N/A'),
        'estacionaria': resultados.get('estacionariedad', {}).get('es_estacionaria', False),
        'mape': resultados.get('evaluacion', {}).get('MAPE', 'N/A')
    }
    pd.Series(resumen_modelado).to_csv(f"data/modelos/resumen_{nombre_serie.replace(' ', '_').lower()}.csv")
    print(f"✓ Pipeline de modelado completado para {nombre_serie}")
    return resultados

series_a_modelar = {
    "Fondo Moderado Skandia": df_skandia_pensiones_moderado,
    "Fondo Conservador Porvenir": df_porvenir_pensiones_conservador,
    "Cesantías Largo Plazo Colfondos": df_colfondos_cesantias_largo_plazo
}

resultados_modelado = {}
for nombre_serie, df_serie in series_a_modelar.items():
    if len(df_serie) > 100:
        try:
            resultados = pipeline_modelado_completo(df_serie, nombre_serie)
            resultados_modelado[nombre_serie] = resultados
        except Exception as e:
            print(f"✗ Error en modelado de {nombre_serie}: {e}")
    else:
        print(f"⚠️  Serie {nombre_serie} muy corta para modelado ({len(df_serie)} registros)")

if resultados_modelado:
    comparacion_modelos = []
    for nombre, resultados in resultados_modelado.items():
        if resultados.get('mejor_modelo'):
            comparacion_modelos.append({
                'Serie': nombre,
                'Mejor Modelo': f"ARIMA{resultados['mejor_modelo']['orden']}",
                'AIC': resultados['mejor_modelo']['metricas']['aic'],
                'Estacionaria': resultados['estacionariedad']['es_estacionaria'],
                'MAPE (%)': resultados.get('evaluacion', {}).get('MAPE', 'N/A')
            })
    if comparacion_modelos:
        df_comparacion = pd.DataFrame(comparacion_modelos)
        print("
📊 COMPARACIÓN DE MODELOS:")
        print(df_comparacion.to_string(index=False))
        df_comparacion.to_csv("data/modelos/comparacion_modelos.csv", index=False)

        mejor_modelo = df_comparacion.loc[df_comparacion['AIC'].idxmin()]
        print(f"
• Mejor modelo general: {mejor_modelo['Serie']} ({mejor_modelo['Mejor Modelo']})")
        print(f"• AIC más bajo: {mejor_modelo['AIC']:.2f}")
        print("
💡 Recomendaciones:")
        print("1. Para series estacionarias considerar modelos ARMA.")
        print("2. Para series no estacionarias explorar SARIMA estacional.")
        print("3. Revisar outliers cuando el MAPE sea alto.")
        print("4. Comparar con modelos de machine learning.")

Path("data/graficas_modelado").mkdir(parents=True, exist_ok=True)
for nombre_serie, resultados in resultados_modelado.items():
    if resultados.get('mejor_modelo'):
        try:
            plt.figure(figsize=(15, 10))
            plt.subplot(2, 2, 1)
            serie_original = resultados['serie_original']
            modelo = resultados['mejor_modelo']['modelo']
            plt.plot(serie_original.index, serie_original.values, label='Original', alpha=0.7)
            plt.plot(modelo.fittedvalues.index, modelo.fittedvalues, label='Ajustado', alpha=0.8)
            plt.title(f'Serie Original vs Ajustada
{nombre_serie}')
            plt.legend()
            plt.xticks(rotation=45)

            plt.subplot(2, 2, 2)
            residuos = modelo.resid
            plt.plot(residuos.index, residuos.values)
            plt.title('Residuos del Modelo')
            plt.axhline(y=0, color='r', linestyle='--')
            plt.xticks(rotation=45)

            plt.subplot(2, 2, 3)
            plt.hist(residuos.dropna(), bins=50, alpha=0.7, density=True)
            plt.title('Distribución de Residuos')
            plt.xlabel('Residuos')
            plt.ylabel('Densidad')

            plt.subplot(2, 2, 4)
            if 'pronostico_futuro' in resultados:
                pronostico = resultados['pronostico_futuro']
                ultimos_30 = serie_original.tail(30)
                plt.plot(ultimos_30.index, ultimos_30.values, label='Últimos 30 días', color='blue')
                plt.plot(pronostico.index, pronostico.values, label='Pronóstico 30 días', color='red', linestyle='--')
                plt.legend()
            plt.title('Pronóstico a 30 días')
            plt.xticks(rotation=45)

            plt.tight_layout()
            plt.savefig(f"data/graficas_modelado/resultados_{nombre_serie.replace(' ', '_').lower()}.png", dpi=300, bbox_inches='tight')
            plt.close()
            print(f"✓ Gráficas guardadas para: {nombre_serie}")
        except Exception as e:
            print(f"✗ Error generando gráficas para {nombre_serie}: {e}")

print("
PIPELINE DE MODELADO COMPLETADO ✓")
print(f"Series modeladas: {len(resultados_modelado)}")
print(f"Modelos generados: {sum(1 for r in resultados_modelado.values() if r.get('mejor_modelo'))}")
print("Resultados guardados en data/modelos/ y gráficas en data/graficas_modelado/")


SyntaxError: unterminated f-string literal (detected at line 8) (302685631.py, line 8)